In [5]:
import pandas as pd 
import os
import numpy as np
import pipreqs

In [6]:
print(pd.__version__)

1.3.4


In [25]:
# !pip install pipreqs

  Using cached pipreqs-0.4.11-py2.py3-none-any.whl (32 kB)
  Using cached yarg-0.1.9-py2.py3-none-any.whl (19 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl


In [2]:
# !pip install fastparquet

  Created wheel for thrift: filename=thrift-0.15.0-cp39-cp39-win_amd64.whl size=176708 sha256=89cc315a6eb81b594cd2722c6ff2f1335d3cb6c87e43b73b195e30d4b94dc001
  Stored in directory: c:\users\alaek\appdata\local\pip\cache\wheels\00\41\cd\f65d329f7cc147ce79cc833b11b94f7bea7329df0ad9d81629
Successfully built thrift


In [20]:
path_test = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Test/X_stations/X_station_test.csv'
path_train = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Train/X_stations/X_station_train.csv'
coords_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/other/stations_coordinates.csv'
output_data = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/output_data'

In [23]:
train = pd.read_csv(path_train)
train.head()

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4


In [5]:
train.rename(columns={"ff":"wind_speed","t":"temp","hu":"humidity","dd":"wind_direction","td":"dew_point_temp"}, inplace=True)

In [7]:
## Reading the 325 stations accross the NW of France
coords = pd.read_csv(coords_path)
coords.head()

,number_sta,lat,lon,height_sta
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0


In [8]:
train = train.merge(coords, on=['number_sta'], how='left')
train.head()

,number_sta,date,wind_speed,temp,dew_point_temp,humidity,wind_direction,precip,Id,lat,lon,height_sta
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,49.334,-0.431,2.0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,49.334,-0.431,2.0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,49.334,-0.431,2.0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,49.334,-0.431,2.0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,49.334,-0.431,2.0


In [9]:
train.dtypes

number_sta          int64
date               object
wind_speed        float64
temp              float64
dew_point_temp    float64
humidity          float64
wind_direction    float64
precip            float64
Id                 object
lat               float64
lon               float64
height_sta        float64
dtype: object

In [10]:
train.height_sta = train.height_sta.astype('int16')

In [11]:
test = pd.read_csv(path_test)
test.head()

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8


In [12]:
test.rename(columns={"ff":"wind_speed","t":"temp","hu":"humidity","dd":"wind_direction","td":"dew_point_temp"}, inplace=True)

In [13]:
test.head()

,wind_direction,humidity,dew_point_temp,temp,wind_speed,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8


In [14]:
def spliting(string, index):
    liste = string.split("_")
    return liste[index]

In [15]:
test['number_sta'] = test['Id'].apply(lambda x : spliting(x, 0))
test.head()

,wind_direction,humidity,dew_point_temp,temp,wind_speed,precip,month,Id,number_sta
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,14047002
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,14047002
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,14047002
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,14047002
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,14047002


In [16]:
test.dtypes

wind_direction    float64
humidity          float64
dew_point_temp    float64
temp              float64
wind_speed        float64
precip            float64
month               int64
Id                 object
number_sta         object
dtype: object

In [17]:
test.number_sta = test.number_sta.astype(int)

In [18]:
test = test.merge(coords, on=['number_sta'], how='left')
test.head()

,wind_direction,humidity,dew_point_temp,temp,wind_speed,precip,month,Id,number_sta,lat,lon,height_sta
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,14047002,49.275,-0.712,60.0
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,14047002,49.275,-0.712,60.0
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,14047002,49.275,-0.712,60.0
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,14047002,49.275,-0.712,60.0
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,14047002,49.275,-0.712,60.0


In [19]:
test.height_sta = test.height_sta.astype('int16')

In [21]:
test.to_parquet(os.path.join(output_data,"X_test_stations_prepared.gzip"), compression='gzip', engine='fastparquet')

In [ ]:
# !cp X_test_stations_prepared.gzip /content/drive/MyDrive/meteo_france/test_stations_data/Data_preparation_results

In [22]:
train.to_parquet(os.path.join(output_data,"X_train_stations_prepared.gzip"), compression='gzip', engine='fastparquet')

In [ ]:
# !cp X_train_stations_prepared.gzip /content/drive/MyDrive/meteo_france/train_stations_data/Data_preparation_results